In [1]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from scipy.stats import kendalltau
from sklearn.metrics import ndcg_score
from openai import OpenAI

In [2]:
corpus = pd.read_json("corpus.jsonl", lines=True)
queries = pd.read_json("queries.jsonl", lines=True)
corpus.set_index("_id", inplace=True)
queries.set_index("_id", inplace=True)
top_qrels = pd.read_csv("top_qrels.tsv", sep="\t")
top_qrels

,query-id,corpus-id,e5-score,score
0,0,599,83.478034,1
1,0,91397,83.293903,0
2,0,140938,83.293903,0
3,0,9725,83.293903,0
4,0,598,83.043307,1
...,...,...,...,...
10675,558,188549,84.378934,0
10676,558,188527,84.362769,0
10677,558,188537,84.205627,0
10678,558,188543,84.190488,0


In [3]:
prompt_name = "gemini_fact"
full_name = f"deepseek_{prompt_name}.tsv"

In [4]:
deepseek_df = pd.read_csv(full_name, sep="\t")

In [5]:
def merge_scores_llm(e5_df, llm):
  merged = pd.merge(
        e5_df,
        llm,
        on=['query-id', 'corpus-id'],
        how='left'
    )

  del merged["human-score"]
  
  return merged
ftop_qrels = merge_scores_llm(top_qrels, deepseek_df)

In [6]:
class DeepSeekJudge:
  def __init__(self):
    self.client = OpenAI(api_key="sk-21cb3976392646d687940daed0ca3650", base_url="https://api.deepseek.com")
  def generate_judgement(self, query, passage):
    system_message = """
Ты — строгий эксперт по проверке фактов и оценке семантической релевантности. Твоя задача — проанализировать запрос и отрывок текста, чтобы определить, насколько отрывок подтверждает или содержит информацию из запроса.

Используй следующую шкалу для оценки (целое число от 0 до 2):

[2] Полностью релевантный: Отрывок содержит исчерпывающую информацию, подтверждающую весь факт из запроса. Смысл запроса полностью раскрыт в тексте.
[1] Частично релевантный: Отрывок касается темы запроса, упоминает ключевые сущности или содержит часть факта, но не дает полного подтверждения или ответа.
[0] Нерелевантный: Отрывок не имеет отношения к запросу, не содержит упоминаний ключевых объектов или противоречит контексту.

ВАЖНО:
1. Твой ответ должен быть СТРОГО в формате JSON.
2. Используй ключ "score".
3. Значение должно быть только числом: 0, 1 или 2.
4. Не пиши никаких пояснений, комментариев или вводных слов. Только JSON.

Пример формата ответа:
{"score": 1}
"""
    user_message_template = f"""
Оцени релевантность следующей пары текстов, используя заданные критерии.

ЗАПРОС:
{query}

ОТРЫВОК:
{passage}

Выведи оценку в JSON формате: {{"score": N}}
"""
    response = self.client.chat.completions.create(
      model="deepseek-chat",
      messages=[
          {"role": "system", "content": system_message},
          {"role": "user", "content": user_message_template},
      ],
      stream=False,
      response_format={
        'type': 'json_object'
      }
    )
    return response.choices[0].message.content

In [7]:
judge = DeepSeekJudge()

In [8]:
results = []
for idx, row in tqdm(ftop_qrels.iterrows(), total=len(ftop_qrels)):
  true_score = row["score"]
  llm_score = row["llm-score"]
  if pd.isna(llm_score):
    query = queries.loc[row["query-id"]]["text"]
    response = corpus.loc[row["corpus-id"]]["text"]
    res = judge.generate_judgement(query, response)
    res = json.loads(res)
    llm_score = res["score"]
  result = {
    "llm-score": llm_score,
    "human-score": true_score,
    "query-id": row["query-id"],
    "corpus-id": row["corpus-id"]
  }
  results.append(result)

100%|██████████| 10680/10680 [3:58:28<00:00,  1.34s/it] 


In [9]:
results_df = pd.DataFrame(results)
from sklearn.metrics import cohen_kappa_score
llm = results_df["llm-score"].to_numpy(int)
human = results_df["human-score"].to_numpy(int)
cohen_kappa_score(llm, human)

0.30646262912063527

In [10]:
import numpy as np
import metrics


print("NDCG: ", metrics.calc_ndcg(results_df))
print("NRMSE: ", metrics.calc_nrmse(results_df))
print("NMAE: ", metrics.calc_nmae(results_df))
print("kendalltau: ", metrics.calc_kendalltau(results_df))
print("RBO: ", metrics.calc_rbo(results_df))

NDCG:  0.8032047888429822
NRMSE:  0.332149113664127
NMAE:  0.1821629213483146
kendalltau:  0.5127738609288442
RBO:  0.6092175609049182


In [11]:
pd.DataFrame(results_df).to_csv(f"rerank_{full_name}", sep='\t', index=False)